## Week 3 Peer Reviewed Assignment

In [ ]:
# Notebook for the final IBM Data Science Course from Coursera

In [1]:
# Import neccessary libraries

In [1]:

# library to handle data in a vectorized manner

import numpy as np

import pandas as pd
from pandas import DataFrame

# library to handle JSON files
import json 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

%matplotlib inline

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## Webscraping using BeautifulSoup package

In [2]:
# Import the BeautifulSoup package for web scraping
from bs4 import BeautifulSoup

In [3]:
#Scrapethe wikipedia page for getting postal code of Canada
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")


In [4]:
#parsing the data
soup = BeautifulSoup(res.content,'html.parser')


In [23]:
#exploring the parsed html document
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"cb17fdae-6f3e-4e21-b25b-c0c99cf0d183","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":979555370,"wgRevisionId":979555370,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communicati

In [5]:
#Find the right table and converting the data to a dataframe
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]


In [6]:
#Inspect the newly created dataframe
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [19]:
duplicates = df.pivot_table(index=['Postal Code'], aggfunc='size' )
print (duplicates)

Postal Code
M1A    1
M1B    1
M1C    1
M1E    1
M1G    1
      ..
M9V    1
M9W    1
M9X    1
M9Y    1
M9Z    1
Length: 180, dtype: int64


In [20]:
print(duplicates)

Postal Code
M1A    1
M1B    1
M1C    1
M1E    1
M1G    1
      ..
M9V    1
M9W    1
M9X    1
M9Y    1
M9Z    1
Length: 180, dtype: int64


In [22]:
df.duplicated(subset=['Postal Code'])

0      False
1      False
2      False
3      False
4      False
       ...  
175    False
176    False
177    False
178    False
179    False
Length: 180, dtype: bool

In [23]:
df['Postal Code'].nunique()

180

,Postal Code,Borough,Neighbourhood


In [16]:
#Alternative way of creating the dataframe (a long winded way. I'm only experimenting here!)

postalCodeList = []
boroughList = []
neighborhoodList = []


for row in soup.find('table').find_all('tr'):
    cells = []
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
            cells.append(column.get_text())
            column_marker += 1
          
    if(len(cells) > 0):
        postalCodeList.append(cells[0].rstrip('\n'))
        boroughList.append(cells[1].rstrip('\n'))
        neighborhoodList.append(cells[2].rstrip('\n'))
        
    
   
     

In [17]:
# creating the dataframe (in a alternative way) with the columns we want using pandas
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

In [9]:
df[df['Borough'] == 'Not assigned'].count()

Postal Code      77
Borough          77
Neighbourhood    77
dtype: int64

In [29]:
df[df['Neighbourhood'] == 'Not assigned'].count()

Postal Code      77
Borough          77
Neighbourhood    77
dtype: int64

In [31]:
#exploring the alternative dataframe
toronto_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Dropping rows in Borough column with 'Not assigned' values  

In [32]:
#Dropping rows in Borough column with 'Not assigned' values  

df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)

In [34]:
#checking if all the 'Not assigned' rows have indeed been dropped
df[df['Borough'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [35]:
df[df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [76]:
df.info

<bound method DataFrame.info of     Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177       

In [77]:
df.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

In [78]:
#removing any white spaces from the column names
df.columns = df.columns.str.strip()

## Combining Neighbourhoods in a single row with the same postal code 

In [79]:
#combining Neighbourhoods in a single row with the same postal code 
df_group = df.groupby(["Postal Code", "Borough"], as_index=False)['Neighbourhood'].agg(lambda x: ','.join(x))

In [80]:
df_group

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
...,...,...,...
175,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W,Etobicoke,"Northwest, West Humber - Clairville"
177,M9X,Not assigned,Not assigned
178,M9Y,Not assigned,Not assigned


In [81]:
pd.Series(df['Postal Code']).is_unique

True

In [82]:
df[df['Postal Code'] == 'M5A']

,Postal Code,Borough,Neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [83]:
duplicated =df[df.duplicated()]

In [84]:
duplicated

,Postal Code,Borough,Neighbourhood


In [87]:
df[df['Neighbourhood'] == 'Not assigned'].count()

Postal Code      77
Borough          77
Neighbourhood    77
dtype: int64

In [88]:
df[df['Borough'] == 'Not assigned'].count()

Postal Code      77
Borough          77
Neighbourhood    77
dtype: int64

In [89]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
for a in df_group['Borough']:
    if a != 'Not assigned':
        Borough_na = a
        #print(a)
        for i in df_group['Neighbourhood']:
            #print(i)
            y = 0
            #neighbourhood_na = i
            #print(neighbourhood_na)
            if i == 'Not assigned':
                #print("I am here")
                neighbourhood_na = Borough_na
                #print(i)
                df_group.loc[y]['Neighbourhood'] = neighbourhood_na
                print(df_group.loc[y]['Neighbourhood'])
                y += 1
                

            
    


Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scar

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
No

North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York

East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Cen

Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toron

Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central 

York
York
York
York
York
York
York
York
York
York
York
York
York
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto


Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke


Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke
Etobicoke


## merging with geospatial data and reshaping the dataframe as asked in the assignment question

In [90]:
df_group

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Etobicoke
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
...,...,...,...
175,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W,Etobicoke,"Northwest, West Humber - Clairville"
177,M9X,Not assigned,Not assigned
178,M9Y,Not assigned,Not assigned


In [91]:
#reshaping the dataframe in the way it's asked in the assignment
final_df_test = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood']) 
print(final_df_test) 
  

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


In [92]:
#reshaping the dataframe in the way it's asked in the assignment (continued)
post_code_list = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']

In [93]:
#reshaping the dataframe in the way it's asked in the assignment (continued)
for i in post_code_list:
    final_df_test = final_df_test.append(df_group_merged[df_group_merged['Postal Code'] ==  i]).reset_index(drop = True)

NameError: name 'df_group_merged' is not defined

In [94]:
#Exploring the re-shaped dataframe
final_df_test

,Postal Code,Borough,Neighbourhood


In [95]:
# using the Geospatial_Coordinates.csv to get respective coordinates for each postal code
df_geo = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
#Exploring the geospatial dataframe with the grouped dataframe 
df_group_merged = pd.merge(df_group,df_geo)

In [104]:
df_group_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [99]:
#Creating a new dataframe to display the data in the way asked in the assignment
final_df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood']) 
print(final_df) 

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


In [100]:
#reshaping the dataframe in the way it's asked in the assignment
for i in post_code_list:
    final_df = final_df.append(df_group_merged[df_group_merged['Postal Code'] ==  i]).reset_index(drop = True)

In [101]:
#checking if the dataframe matches the what was asked in the assignment 
final_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [102]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [103]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [105]:
CLIENT_ID = 'DJ3WT0BU4G01WASVHSNKMJME24MVFZNPXUGULBP30MRNXZAL' # your Foursquare ID
CLIENT_SECRET = 'VHRAWJ3PLFPUSI5HEPHUXQSQU510SFS5PSVG05VCGFXEFRKV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DJ3WT0BU4G01WASVHSNKMJME24MVFZNPXUGULBP30MRNXZAL
CLIENT_SECRET:VHRAWJ3PLFPUSI5HEPHUXQSQU510SFS5PSVG05VCGFXEFRKV


In [111]:
neighborhood_latitude = df_group_merged.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_group_merged.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_group_merged.loc[0, 'Neighbourhood'] # neighborhood name

In [113]:
search_query = 'Indian'
radius = 500
print(search_query + ' .... OK!')

Indian .... OK!


In [118]:
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DJ3WT0BU4G01WASVHSNKMJME24MVFZNPXUGULBP30MRNXZAL&client_secret=VHRAWJ3PLFPUSI5HEPHUXQSQU510SFS5PSVG05VCGFXEFRKV&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [119]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd5064e383e6d764f30aacf'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [ ]:
43.657952	-79.387383